<a href="https://colab.research.google.com/github/kfuentestorres/Master-BI-BDA-Project-Work/blob/master/Tabella_Analisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [13]:
data = pd.read_csv("ft_WebNews.csv")
news = data.copy()

campi che serviranno per individuare le parole legate alle fake news

In [14]:
news['words_sottotitolo'] = [''.join(y for y in x.replace("'", " ").replace("’", " ").lower() if y.isalpha() or y.isspace()) for x in news['sottotitolo']]
news['words_titolo'] = [''.join(y for y in x.replace("'", " ").replace("’", " ").lower() if y.isalpha() or y.isspace()) for x in news['titolo']]

campi che serviranno per fare analisi

In [15]:
news['caratteri_sottotitolo'] = ["non presente" if x == "non presente" else "<100" if len(x) < 100 else '>=100' for x in news['sottotitolo']]
news['caratteri_testo'] = ['<500' if len(x) < 500 else '<1000' if len(x) < 1000 else '<1500' if len(x) < 1500 else '>=1500' for x in news['body']]
news['pnumero_paragrafi'] = ['<2' if x < 2 else '<6' if x < 6 else '>=6' for x in news['paragrafi']]
news['ora'] = ["non presente" if x == "non presente" else "presente" for x in news['time']]
news['tipologia_autore'] = ["non presente" if x == "non presente" else "sigle" if '.' in x and len(x) < 16 else "nome completo" for x in news['autore']]

campi dei parametri che serviranno per il calcolo dell'indice di attendibilità

In [16]:
news['p1'] = [0 if x == "non presente" else 1 if len(x) < 100 else 2 for x in news['sottotitolo']]
news['p2'] = [0 if len(x) < 500 else 1 if len(x) < 1000 else 2 if len(x) < 1500 else 3 for x in news['body']]
news['p3'] = [0 if x < 2 else 1 if x < 6 else 2 for x in news['paragrafi']]
news['p4'] = [0 if x == "non presente" else 1 for x in news['time']]
news['p5'] = [0 if x == "non presente" else 1 for x in news['foto']]
news['p6'] = [0 if x == "non presente" else 1 for x in news['video']]
news['p7'] = [0 if x == "non presente" else 1 if '.' in x and len(x) < 16 else 2 for x in news['autore']]

parole legate alle fake news e cluster di parole associate

In [17]:
w1 = ' laboratorio '
c1 = [' creato ', ' wuhan ', ' nato ', ' produzione ', ' uscito ', ' esistenza ', ' creata ', ' diffuso ']
w2 = ' vitamina '
c2 = [' guarire ', ' prevenire ', ' placare ', ' ridurre ']
w3 = ' candeggina '
c3 = [' diluire ', ' distruggerlo ', ' uccidere ', ' ucciderlo ', ' difendersi ']
w4 = ' 5g '
c4 = [' correlazione ', ' correlazioni ']
w5 = ' alcol '
c5 = [' eliminarlo ', ' proteggere ', ' difendersi ']
w6 = ' metanolo '
c6 = [' efficace ', ' rimedio ']

In [18]:
news['p8'] = [0 if w1 in x and any(y in x for y in c1) \
              or w2 in x and any(y in x for y in c2) \
              or w3 in x and any(y in x for y in c3) \
              or w4 in x and any(y in x for y in c4) \
              or w5 in x and any(y in x for y in c5) \
              or w6 in x and any(y in x for y in c6) \
              else 1 for x in news['words_sottotitolo']]

news['p9'] = [0 if w1 in x and any(y in x for y in c1) \
              or w2 in x and any(y in x for y in c2) \
              or w3 in x and any(y in x for y in c3) \
              or w4 in x and any(y in x for y in c4) \
              or w5 in x and any(y in x for y in c5) \
              or w6 in x and any(y in x for y in c6) \
              else 2 for x in news['words_titolo']]

creazione campo per individuare le fake news trovate e campo contenente i punteggi degli articoli

In [19]:
news['FakeNews_flag'] = np.where((news.p8 == 0) | (news.p9 == 0), 'S', 'N')

news['indicatore'] = round((news['p1'] + news['p2'] + news['p3'] + news['p4'] + news['p5'] + news['p6'] + news['p7'] + news['p8'] + news['p9']) * 5 / 15, 2)

salvataggio della tabella finale

In [20]:
news = news.drop(['_id', 'words_sottotitolo', 'words_titolo', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7',  'p8',  'p9'], 1)

news.head()

,id,fonte,titolo,sottotitolo,body,paragrafi,date,time,autore,foto,video,share,url,caratteri_sottotitolo,caratteri_testo,pnumero_paragrafi,ora,FakeNews_flag,indicatore
0,21689489,Libero,"Coronavirus, i farmacisti sotto assedio: ""Così...",non presente,"Le farmacie, in piena emergenza coronavirus, v...",8,31/03/2020,non presente,Paola Natali,presente,non presente,presente,https://www.liberoquotidiano.it/news/italia/21...,non presente,>=1500,>=6,non presente,N,3.67
1,21689486,Libero,"Coronavirus, una ""pandemia ogni 7 anni"". Dalla...",non presente,"Le pandemie sono sempre più frequenti, e il mo...",2,31/03/2020,non presente,non presente,presente,non presente,presente,https://www.liberoquotidiano.it/news/scienze-t...,non presente,<1500,<6,non presente,N,2.33
2,21689484,Libero,"Coronavirus, stress, alimentazione e sonno. L'...",non presente,Tempo di stare a casa ed è possibile trasforma...,8,31/03/2020,non presente,non presente,presente,non presente,presente,https://www.liberoquotidiano.it/news/scienze-t...,non presente,>=1500,>=6,non presente,N,3.00
3,21689321,Libero,"Coronavirus, seduta segreta al Senato senza di...",non presente,"""Seduta segreta al Senato"". Fonti della Lega d...",2,31/03/2020,non presente,non presente,presente,non presente,presente,https://www.liberoquotidiano.it/news/politica/...,non presente,<1000,<6,non presente,N,2.00
4,21688952,Libero,Maria Giovanna Maglie e Socci a tenaglia sul g...,non presente,Chi pagherà per tutto questo? Se lo chiedono A...,2,31/03/2020,non presente,non presente,presente,non presente,presente,https://www.liberoquotidiano.it/news/personagg...,non presente,<1000,<6,non presente,N,2.00


In [21]:
news.to_csv("ds_webnews.csv")